## Dataframe operations
Spark DataFrames allow operations similar to pandas Dataframes. We demonstrate some of those.

For more, see the [official guide](https://spark.apache.org/docs/latest/sql-programming-guide.html) and [this article](https://www.analyticsvidhya.com/blog/2016/10/spark-dataframe-and-operations/)

## But first, some important details
Spark on datahub can run in two modes:

* **local mode** which means that it is run on the same computer as the head node. This is convenient for small jobs and for debugging. Can also be done on your laptop.

* **remote mode** in which the head node and the worker nodes are separate. This mode requires that the spark cluster is up and running. In this case the full resources of the clusters are available. This mode is useful when processing large jobs.

In [1]:
import os
import sys

import pyspark
from pyspark import SparkContext
from lib import sparkConfig

172.17.0.2


In [2]:
%%time
#sc.stop()  # uncomment if sparkContex already exists
sc = sc = SparkContext('local', 'test')
#SparkContext() #conf=sparkConfig.conf)
sc

CPU times: user 13.3 ms, sys: 6.8 ms, total: 20.1 ms
Wall time: 1.73 s


<SparkContext master=local appName=test>

In [3]:
%%time
from pyspark import SparkContext
from pyspark.sql import SQLContext
from pyspark.sql.types import Row, StructField, StructType, StringType, IntegerType,BinaryType

sqlContext = SQLContext(sc)
sqlContext

CPU times: user 7.07 ms, sys: 3.95 ms, total: 11 ms
Wall time: 109 ms


/usr/local/spark/python/pyspark/sql/context.py:77: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(


In [8]:
%%time
datasets='/datasets/weather/datasets/'
parquet_path=datasets+'/weather-parquet/'
df=sqlContext.read.parquet(parquet_path)

CPU times: user 920 µs, sys: 1.95 ms, total: 2.87 ms
Wall time: 105 ms


In [9]:
df.printSchema()

root
 |-- Station: string (nullable = true)
 |-- Measurement: string (nullable = true)
 |-- Year: integer (nullable = true)
 |-- Values: binary (nullable = true)



In [10]:
%%time
print(df.count())
df.show(1)

12720632
+-----------+-----------+----+--------------------+
|    Station|Measurement|Year|              Values|
+-----------+-----------+----+--------------------+
|AG000060390|       TAVG|2022|[79 00 6C 00 66 0...|
+-----------+-----------+----+--------------------+
only showing top 1 row

CPU times: user 4.39 ms, sys: 5.78 ms, total: 10.2 ms
Wall time: 4.84 s


### .describe()
The method `df.describe()` computes five statistics for each column of the dataframe `df`.

The statistics are: **count, mean, std, min,max**

You get the following man page using the command `df.describe?`

```
Signature: df.describe(*cols)
Docstring:
Computes statistics for numeric and string columns.

This include count, mean, stddev, min, and max. If no columns are
given, this function computes statistics for all numerical or string columns.

.. note:: This function is meant for exploratory data analysis, as we make no
    guarantee about the backward compatibility of the schema of the resulting DataFrame.

>>> df.describe(['age']).show()
+-------+------------------+
|summary|               age|
+-------+------------------+
|  count|                 2|
|   mean|               3.5|
| stddev|2.1213203435596424|
|    min|                 2|
|    max|                 5|
+-------+------------------+
>>> df.describe().show()
+-------+------------------+-----+
|summary|               age| name|
+-------+------------------+-----+
|  count|                 2|    2|
|   mean|               3.5| null|
| stddev|2.1213203435596424| null|
|    min|                 2|Alice|
|    max|                 5|  Bob|
+-------+------------------+-----+

.. versionadded:: 1.3.1
File:      ~/spark-2.2.1-bin-hadoop2.7/python/pyspark/sql/dataframe.py
Type:      method
```

In [11]:
df.describe().select('Station','Measurement','Year').show() 

+-----------+-----------+------------------+
|    Station|Measurement|              Year|
+-----------+-----------+------------------+
|   12720632|   12720632|          12720632|
|       null|       null|1977.4126412901496|
|       null|       null|31.931757686816614|
|ACW00011604|       ACMC|              1764|
|ZI000067991|       WV20|              2022|
+-----------+-----------+------------------+



#### groupby and agg
The method `.groupby(col)` groups rows according the value of the column `col`.  
The method `.agg(spec)` computes a summary for each group as specified in `spec`

In [12]:
df.groupby('Measurement').agg({'Year': 'min', 'Station':  'count'}).show()

+-----------+--------------+---------+
|Measurement|count(Station)|min(Year)|
+-----------+--------------+---------+
|       WESD|        123619|     1952|
|       PGTM|         31995|     1948|
|       AWDR|           730|     1996|
|       SX13|            60|     1982|
|       WT07|         19790|     1891|
|       WT13|          7659|     1996|
|       SX33|           524|     1982|
|       EVAP|         27035|     1893|
|       SN53|           201|     1982|
|       WT10|          2487|     1886|
|       SN35|            59|     2005|
|       TMIN|       1388531|     1764|
|       WT21|          2434|     1996|
|       DATX|          7766|     1863|
|       WT14|         38509|     1851|
|       WT19|          4928|     1992|
|       SX02|          3373|     1982|
|       MDPR|        515120|     1832|
|       WT09|         30516|     1852|
|       SX51|           178|     1982|
+-----------+--------------+---------+
only showing top 20 rows



In [9]:
# THis command will load the python module that defines the SQL functions
#%load ls ~/spark-latest/python/pyspark/sql/functions.py

### Using SQL queries on DataFrames

There are two main ways to manipulate  DataFrames:

#### Imperative manipulation
Using python methods such as `.select` and `.groupby`.
* Advantage: order of operations is specified.
* Disrdavantage : You need to describe both **what** is the result you want and **how** to get it.

#### Declarative Manipulation (SQL)
* Advantage: You need to describe only **what** is the result you want.
* Disadvantage: SQL does not have primitives for common analysis operations such as **covariance**

### Using sql commands on a dataframe
Spark supports a [subset](https://spark.apache.org/docs/latest/sql-programming-guide.html#supported-hive-features) of the Hive SQL query language.

For example, You can use [Hive `select` syntax](https://cwiki.apache.org/confluence/display/Hive/LanguageManual+Select) to select a subset of the rows in a dataframe.

To use sql on a dataframe you need to first `register` it as a `TempTable`.

for variety, we are using here a small dataframe loaded from a JSON file.

### Counting the number of occurances of each measurement, imparatively

In [13]:
%%time
L=df.groupBy('measurement').count().collect()
#L is a list of Rows (collected DataFrame)

CPU times: user 7.17 ms, sys: 1.21 ms, total: 8.38 ms
Wall time: 1.82 s


In [14]:
D=[(e.measurement,e['count']) for e in L]
print('The most common mesurements')
sorted(D,key=lambda x:x[1], reverse=True)[:6]

The most common mesurements


[('PRCP', 3256604),
 ('TMIN', 1388531),
 ('TMAX', 1384761),
 ('SNOW', 1257297),
 ('SNWD', 1208442),
 ('TOBS', 536759)]

In [15]:
print('The most rare mesurements')
sorted(D,key=lambda x:x[1], reverse=False)[:6]

The most rare mesurements


[('SN57', 1), ('SX15', 2), ('SX57', 2), ('SX17', 2), ('SN14', 2), ('SX14', 3)]

### Counting the number of occurances of each measurement, declaratively.

#### Registrering a dataframe as a table in a database

In order to apply SQL commands to a dataframe, it has to first be registered as a table in the database managed by sqlContext.

In [16]:
sqlContext.registerDataFrameAsTable(df,'weather') #using older sqlContext instead of newer (V2.0) sparkSession

In [17]:
%%time
query="""
SELECT measurement,COUNT(measurement) AS count,
                   MIN(year) AS MinYear 
FROM weather  
GROUP BY measurement 
ORDER BY count DESC
"""
print(query)
sqlContext.sql(query).show(5)


SELECT measurement,COUNT(measurement) AS count,
                   MIN(year) AS MinYear 
FROM weather  
GROUP BY measurement 
ORDER BY count DESC

+-----------+-------+-------+
|measurement|  count|MinYear|
+-----------+-------+-------+
|       PRCP|3256604|   1781|
|       TMIN|1388531|   1764|
|       TMAX|1384761|   1764|
|       SNOW|1257297|   1840|
|       SNWD|1208442|   1857|
+-----------+-------+-------+
only showing top 5 rows

CPU times: user 3.83 ms, sys: 2.37 ms, total: 6.19 ms
Wall time: 1.97 s


#### Performing a map command
* Dataframes do not support `map` and `reduce` operations.
* In order to perform a `map` or `reduce` on a dataframe, you first need to transform it into an RDD.

* This is a quick-and-dirty solution. 
* A better way is to use [built-in sparkSQL functions](https://spark.apache.org/docs/latest/api/sql/index.html)
* Or if you can't find what you need, you can try and create a [User-Defined-function* (UDF)](https://spark.apache.org/docs/latest/sql-ref-functions-udf-scalar.html)

In [18]:
df.rdd.map(lambda row:(row.Station,row.Year)).take(5)

[('AG000060390', 2022),
 ('AGE00147716', 2022),
 ('AGM00060360', 2022),
 ('AGM00060421', 2022),
 ('AGM00060430', 2022)]

#### Aggregations 
* **Aggregation** can be used, in combination with built-in sparkSQL functions 
to compute statistics of a dataframe.
* computation will be fast thanks to combined optimzations with database operations.

* A partial list : `count(), approx_count_distinct(), avg(), max(), min()`

* Of these, the interesting one is `approx_count_distinct()` which uses sampling to get an approximate count fast.

In [19]:
import pyspark.sql.functions as F # used here just for show.

In [20]:
df.agg({'station':'approx_count_distinct'}).show()

+------------------------------+
|approx_count_distinct(station)|
+------------------------------+
|                        128546|
+------------------------------+



#### Approximate Quantile

* Suppose we want to partition the years into 10 ranges
* such that in each range we have approximately the same number of records.
* The method `.approxQuantile` will use a sample to do this for us.

In [21]:
%%time
print('with accuracy 0.1: ',df.approxQuantile('Year', [0.1*i for i in range(1,10)], 0.1))

with accuracy 0.1:  [1764.0, 1948.0, 1961.0, 1973.0, 1980.0, 1990.0, 1997.0, 2006.0, 2022.0]
CPU times: user 8.87 ms, sys: 4.92 ms, total: 13.8 ms
Wall time: 2.27 s


In [22]:
%%time
print('with accuracy 0.001: ',df.approxQuantile('Year', [0.1*i for i in range(1,10)], 0.00001))

with accuracy 0.001:  [1931.0, 1951.0, 1962.0, 1972.0, 1982.0, 1991.0, 2000.0, 2009.0, 2015.0]
CPU times: user 5.42 ms, sys: 1.6 ms, total: 7.03 ms
Wall time: 9.53 s


### Reading rows selectively from Parquet
Suppose we are only interested in snow measurements. We can apply an SQL query directly to the 
parquet files. As the data is organized in columnar structure, we can do the selection efficiently without loading the whole file to memory.

Here the file is small, but in real applications it can consist of hundreds of millions of records. In such cases loading the data first to memory and then filtering it is very wasteful.

In [23]:
query="""SELECT station,measurement,year 
FROM parquet.`%s` 
WHERE measurement=\"SNOW\" """%parquet_path
print(query)
df2 = sqlContext.sql(query)
print(df2.count(),df2.columns)
df2.show(5)

SELECT station,measurement,year 
FROM parquet.`/datasets/weather/datasets//weather-parquet/` 
WHERE measurement="SNOW" 
1257297 ['station', 'measurement', 'year']
+-----------+-----------+----+
|    station|measurement|year|
+-----------+-----------+----+
|BF1SS000005|       SNOW|2022|
|CA001016335|       SNOW|2022|
|CA0010253G0|       SNOW|2022|
|CA00102BFHH|       SNOW|2022|
|CA001036570|       SNOW|2022|
+-----------+-----------+----+
only showing top 5 rows



## Summary

* Dataframes can be manipulated decleratively, which allows for more optimization.
* Dataframes can be stored and retrieved from Parquet files.
* It is possible to refer directly to a parquet file in an SQL query.
* See you next time!

## References
* For an introduction to Spark SQL and Dataframes see: [Spark SQL, DataFrames](https://spark.apache.org/docs/latest/sql-programming-guide.html#spark-sql-dataframes-and-datasets-guide)
* Also [spark-dataframe-and-operations](https://www.analyticsvidhya.com/blog/2016/10/spark-dataframe-and-operations/) from [analyticsvidhya.com](https://www.analyticsvidhya.com)

For complete API reference see
* [SQL programming guide](https://spark.apache.org/docs/latest/sql-programming-guide.html) For Java, Scala and Python (Implementation is first in Scala and Python, later pyspark)
* [pyspark API for the DataFrame class](http://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark.sql.DataFrame) 
* [pyspark API for the pyspark.sql module](http://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark-sql-module)
